本教程介绍如何使用ezdxf库从现有的DXF文档中获取数据。如果您是ezdxf的新用户，请先阅读“入门用法”教程。

加载DXF文件：

In [1]:
import sys
import ezdxf

try:
    doc = ezdxf.readfile("test.dxf")
except IOError:
    print(f"不是DXF文件或者存在I/O错误。")
    sys.exit(1)
except ezdxf.DXFStructureError:
    print(f"DXF文件无效或损坏。")
    sys.exit(2)

这对于来自AutoCAD或BricsCAD等可信来源的DXF文件很好用，对于存在较小或较大缺陷的DXF文件，可以查看`ezdxf.recover`模块。

### 布局

术语“布局”用作与任意实体空间的同义词，该空间可以包含DXF实体，如线、圆、文本等。每个DXF实体只能驻留在一个确切的布局中。

有三种不同的布局类型：

- 模型空间(modelspace)：常见的构建空间
- 纸张空间(paperspace)：用于创建打印布局
- 块布局(block)：可重复使用的元素，每个块都有自己的实体空间

一个DXF文档包括一个确切的模型空间和至少一个纸张空间。 DXF R12只有一个未命名的纸张空间，后来的DXF版本支持多个纸张空间，每个纸张空间都有一个名称。

### 获取模型空间布局

模型空间包含以真实世界单位表示的绘图主题的“真实”世界表示。模型空间的固定名称为“Model”，DXF文档具有特殊的getter方法`modelspace()`。

In [3]:
msp = doc.modelspace()

### 迭代布局中的DXF实体

此代码展示了如何迭代模型空间中的所有DXF实体：

In [5]:
# 帮助函数
def print_entity(e):
    print("LINE on layer: %s\n" % e.dxf.layer)
    print("start point: %s\n" % e.dxf.start)
    print("end point: %s\n" % e.dxf.end)

# 遍历模型空间中的所有实体
msp = doc.modelspace()
for e in msp:
    if e.dxftype() == "LINE":
        print_entity(e)

# 查询模型空间中所有LINE实体
for e in msp.query("LINE"):
    print_entity(e)

LINE on layer: EPLAN311

start point: (213.8145698753751, 222.3204968081464, 0.0)

end point: (217.4625698753752, 222.3204968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 222.3204968081464, 0.0)

end point: (211.3825698753752, 222.3204968081464, 0.0)

LINE on layer: EPLAN533

start point: (213.8145698753751, 221.1044968081464, 0.0)

end point: (213.8145698753751, 213.8084968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 212.5924968081464, 0.0)

end point: (217.4625698753752, 212.5924968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 212.5924968081464, 0.0)

end point: (211.3825698753752, 212.5924968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 122.6084968081464, 0.0)

end point: (217.4625698753752, 122.6084968081464, 0.0)

LINE on layer: EPLAN311

start point: (213.8145698753751, 122.6084968081464, 0.0)

end point: (211.3825698753752, 122.6084968081464, 0.0)

LINE on layer: EPLAN533

st

所有布局对象都支持标准的Python迭代器协议和in操作符。

### 访问实体的DXF属性

`e.dxftype()`方法返回DXF类型，DXF类型始终是大写字符串，例如“LINE”。实体的所有DXF属性都分组在dxf名称空间属性中：

In [22]:
e.dxf.layer  # 实体所在的层，作为字符串

'0'

In [8]:
e.dxf.color  # 实体的颜色编号，作为整数

256

如果未设置DXF属性（DXF属性不存在），则将引发DXFValueError。在这种情况下，get()方法返回默认值或者如果没有指定默认值，则返回None：

In [9]:
p = e.dxf.get("paperspace", 0)  # 如果DXF属性'paperspace'不存在，则实体默认为模型空间。
p

0

或者可以事先检查属性是否存在：

In [13]:
e.dxf.hasattr("paperspace")

False

不支持的DXF属性会引发DXFAttributeError，要检查实体是否支持某个属性，请使用：

In [14]:
e.dxf.is_supported("paperspace")

True

### 获取纸张空间布局

In [20]:
psp = doc.paperspace("布局1")

上述代码检索名为layout0的纸张空间，Paperspace对象的用法与模型空间对象相同。 DXF R12仅提供一个纸张空间，因此在方法调用`doc.paperspace("layout0")`中忽略纸张空间名称或可以省略。对于新的DXF版本，您可以使用`layout_names()`和`layout_names_in_taborder()`方法获取可用布局名称列表。

### 通过查询语言检索实体

Ezdxf为DXF实体提供了灵活的查询语言。所有布局类型都具有query()方法以启动实体查询或使用ezdxf.query.new()函数。

查询字符串是两个查询的组合，首先是所需的实体查询，然后是可选的属性查询，用方括号括起来：“EntityQuery[AttributeQuery]”

实体查询是DXF实体名称的空格分隔列表或特殊名称*。其中*表示所有DXF实体，所有DXF名称都必须大写。 搜索可以通过添加具有感叹号前缀的实体名称（例如！LINE）来排除实体类型。

属性查询用于按其DXF属性选择DXF实体。属性查询是实体查询的补充，仅在实体已匹配实体查询时才会匹配。属性查询是一个布尔表达式，支持运算符：and、or、!。

获取模型空间中的所有LINE实体：

In [23]:
msp = doc.modelspace()
lines = msp.query("LINE")

容器EntityQuery还提供了query()方法，以进一步细化查询，例如检索所有构造层上的所有LINE实体：

In [24]:
construction_lines = lines.query('*[layer=="construction"]')

\*是所有DXF类型的通配符，在这种情况下，您也可以使用LINE而不是\*，在此处起作用，因为源仅包含LINE实体。

这可以作为单个查询执行：

In [25]:
lines = msp.query('LINE[layer=="construction"]')
lines

检索除了具有虚线类型以外的所有构造层模型空间实体的高级查询：

In [28]:
not_dashed_entities = msp.query('*[layer=="construction" & linetype!="DASHED"]')

### EntityQuery的扩展功能

EntityQuery类具有属性和重载运算符，可通过Python特性而不是查询字符串来构建扩展查询。

与上一节中相同的任务，但使用EntityQuery容器的功能：

In [29]:
# 重载的rational operator返回EntityQuery对象而不是bool值！
lines = msp.query("LINES").layer == "construction"
not_dashed_lines = lines.linetype != "DASHED"

### 按groupby()函数检索实体

groupby()函数按用户定义的标准搜索和分组实体。例如，让我们按图层对模型空间中的所有实体进行分组，结果将是一个字典，其中图层名称为dict-key，与匹配此图层的所有实体列表相关联的字典值。

In [30]:
from ezdxf.groupby import groupby
group = groupby(entities=msp, dxfattrib="layer")
group

{'GPDRAW': [<class 'ezdxf.entities.insert.Insert'> INSERT(#5C4),
  <class 'ezdxf.entities.insert.Insert'> INSERT(#612)],
 'EPLAN311': [<class 'ezdxf.entities.line.Line'> LINE(#8AF),
  <class 'ezdxf.entities.line.Line'> LINE(#8B0),
  <class 'ezdxf.entities.line.Line'> LINE(#8B2),
  <class 'ezdxf.entities.line.Line'> LINE(#8B3),
  <class 'ezdxf.entities.line.Line'> LINE(#8B4),
  <class 'ezdxf.entities.line.Line'> LINE(#8B5),
  <class 'ezdxf.entities.line.Line'> LINE(#8B7),
  <class 'ezdxf.entities.line.Line'> LINE(#8B8),
  <class 'ezdxf.entities.line.Line'> LINE(#8B9),
  <class 'ezdxf.entities.line.Line'> LINE(#8BA),
  <class 'ezdxf.entities.line.Line'> LINE(#8BF),
  <class 'ezdxf.entities.line.Line'> LINE(#8C2),
  <class 'ezdxf.entities.line.Line'> LINE(#8C3),
  <class 'ezdxf.entities.line.Line'> LINE(#8C4),
  <class 'ezdxf.entities.line.Line'> LINE(#8C5),
  <class 'ezdxf.entities.line.Line'> LINE(#8C6),
  <class 'ezdxf.entities.line.Line'> LINE(#8C7),
  <class 'ezdxf.entities.line.Line

entities参数可以是任何容器或生成器，该生成器产生DXF实体：

In [31]:
group = msp.groupby(dxfattrib="layer")#按照layer属性分组

for layer, entities in group.items():#遍历字典
    print(f'Layer "{layer}" contains following entities:')
    for entity in entities:
        print(f"    {entity}")
    print("-"*40)

Layer "GPDRAW" contains following entities:
    INSERT(#5C4)
    INSERT(#612)
----------------------------------------
Layer "EPLAN311" contains following entities:
    LINE(#8AF)
    LINE(#8B0)
    LINE(#8B2)
    LINE(#8B3)
    LINE(#8B4)
    LINE(#8B5)
    LINE(#8B7)
    LINE(#8B8)
    LINE(#8B9)
    LINE(#8BA)
    LINE(#8BF)
    LINE(#8C2)
    LINE(#8C3)
    LINE(#8C4)
    LINE(#8C5)
    LINE(#8C6)
    LINE(#8C7)
    LINE(#8C8)
    LINE(#8C9)
    LINE(#8CA)
    LINE(#8CB)
    LINE(#8CC)
    LINE(#8CD)
    LINE(#8CE)
    LINE(#8CF)
    LINE(#8D0)
    LINE(#8D1)
    LINE(#8D2)
    LINE(#8D3)
    LINE(#8D4)
    LINE(#8D5)
    LINE(#8D6)
    LINE(#8D7)
    LINE(#8D8)
    LINE(#8DD)
    LINE(#8DF)
    LINE(#8E0)
    LINE(#8E1)
    LINE(#8E2)
    LINE(#8E3)
    LINE(#8E5)
    LINE(#8E6)
    LINE(#8E7)
    LINE(#8E8)
    LINE(#8E9)
    LINE(#8EA)
    LINE(#8EB)
    LINE(#8EC)
    LINE(#8ED)
    LINE(#8EE)
    LINE(#8EF)
    LINE(#8F0)
    LINE(#8F3)
    LINE(#8F4)
    LINE(#8F5)
    LINE(#

上一个示例展示了如何通过单个DXF属性对实体进行分组。要进行更高级的查询，请创建自定义键函数，该函数接受DXF实体作为参数并返回哈希值作为dict-key或返回None以排除实体。

以下示例显示如何按层和颜色对实体进行分组，字典键是(layer, color)元组，并且与具有匹配DXF属性的实体所匹配的列表相关联的dict-value：

In [32]:
def layer_and_color_key(entity):
    # 返回None以排除来自默认层“0”的实体
    if entity.dxf.layer == "0":
        return None
    return entity.dxf.layer, entity.dxf.color
group = groupby(entities=msp, key=layer_and_color_key)

for (layer, color), entities in group.items():
    print(f'Entities on layer "{layer}" with color {color}:')
for entity in entities:
    print(f" {entity}")
print("-"*40)


Entities on layer "GPDRAW" with color 256:
Entities on layer "EPLAN311" with color 256:
Entities on layer "EPLAN533" with color 1:
Entities on layer "EPLAN533" with color 256:
Entities on layer "EPLAN510" with color 256:
Entities on layer "EPLAN542" with color 256:
Entities on layer "EPLAN311" with color 1:
Entities on layer "EPLAN311" with color 7:
Entities on layer "EPLAN110" with color 7:
Entities on layer "EPLAN100" with color 256:
Entities on layer "EPLAN100" with color 7:
Entities on layer "EPLAN108" with color 7:
Entities on layer "EPLAN108" with color 256:
Entities on layer "EPLAN400" with color 256:
Entities on layer "EPLAN300" with color 256:
Entities on layer "EPLAN501" with color 256:
Entities on layer "EPLAN492" with color 256:
Entities on layer "EPLAN305" with color 256:
Entities on layer "EPLAN420" with color 256:
Entities on layer "EPLAN512" with color 256:
Entities on layer "EPLAN421" with color 256:
Entities on layer "GSuperSymbol" with color 7:
Entities on layer "EPL